Attrition_Flag

In [154]:
import pandas as pd 
DROP_1 = 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'
DROP_2 = 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'

DATA_FILE = "project_data_bank.csv"
data = pd.read_csv(DATA_FILE)
data = data.drop([DROP_1, DROP_2], axis=1)
data

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,2,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,3,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000


In [145]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

FLAG_COL = 'Attrition_Flag'

x = data.drop([FLAG_COL], axis=1)
y = data[FLAG_COL]

# Transform the DataFrame y with a LabelEncoder ('Existing Customer' -> 1 ; Attrited Customer -> 0)
flag_transformer = LabelEncoder()
y = flag_transformer.fit_transform(y)

# Split the dataset into 
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=3)

categorical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['object']]
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols

In [146]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# the dataset has been posted as cleaned, but I still make a SimpleImputer
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [147]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error

model = KNeighborsClassifier()

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(x_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(x_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

MAE: 0.1895360315893386


In [148]:
x_train_rf = pd.get_dummies(x_train)
y_train_rf = pd.get_dummies(y_train)
x_test_rf = pd.get_dummies(x_test)
y_test_rf = pd.get_dummies(y_test)
x_train_rf

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,...,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
4774,719174433,36,1,24,2,1,2,1735.0,0,1735.0,...,0,0,0,0,1,0,1,0,0,0
1523,787509408,26,0,13,6,3,2,2347.0,0,2347.0,...,0,0,0,0,0,1,1,0,0,0
438,721333233,40,4,34,4,2,3,23138.0,1727,21411.0,...,0,0,1,0,0,0,1,0,0,0
1343,778855383,36,5,17,3,1,3,1438.3,0,1438.3,...,0,0,0,0,1,0,1,0,0,0
7767,710740008,57,1,39,3,1,1,1438.3,641,797.3,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6400,715477233,55,1,36,3,2,3,6968.0,1430,5538.0,...,0,0,0,0,0,1,1,0,0,0
9160,795869583,53,3,47,1,3,2,3733.0,1620,2113.0,...,0,0,0,0,1,0,1,0,0,0
9859,714136233,53,4,38,5,2,3,34516.0,0,34516.0,...,1,0,0,0,0,0,1,0,0,0
1688,711226758,51,1,42,3,2,3,4002.0,1669,2333.0,...,0,0,0,0,1,0,1,0,0,0


In [149]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)

rf.fit(x_train_rf, y_train_rf)

rf.score(x_test_rf, y_test_rf)

0.9442250740375123

In [150]:
feature_importances = pd.DataFrame(rf.feature_importances_, index= x_train_rf.columns, columns=['importance']).sort_values('importance', ascending = False)

feature_importances

,importance
Total_Trans_Ct,0.192343
Total_Trans_Amt,0.176785
Total_Revolving_Bal,0.107084
Total_Ct_Chng_Q4_Q1,0.081799
Avg_Utilization_Ratio,0.071144
Total_Amt_Chng_Q4_Q1,0.065202
Total_Relationship_Count,0.044932
Customer_Age,0.032664
Avg_Open_To_Buy,0.029064
Credit_Limit,0.027881


In [151]:
FEATURE_1 = 'Total_Trans_Amt'
FEATURE_2 = 'Total_Trans_Ct'
FEATURE_3 = 'Total_Ct_Chng_Q4_Q1'

x_train_2 = x_train[[FEATURE_1,FEATURE_2, FEATURE_3]]
y_train_2 = y_train
x_test_2 = x_test[[FEATURE_1,FEATURE_2, FEATURE_3]]
y_test_2 = y_test
x_train_2

,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1
4774,2467,35,0.346
1523,2595,56,0.333
438,1612,39,1.438
1343,1975,40,0.600
7767,5140,82,0.519
...,...,...,...
6400,4037,69,1.029
9160,15200,118,0.934
9859,8417,72,0.714
1688,2024,53,1.120


In [152]:
categorical_cols = [cname for cname in x_train_2.columns if x_train_2[cname].dtype in ['object']]
numerical_cols = [cname for cname in x_train_2.columns if x_train_2[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols

# the dataset has been posted as cleaned, but I still make a SimpleImputer
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [153]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(x_train_2, y_train_2)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(x_test_2)

# Evaluate the model
score = mean_absolute_error(y_test_2, preds)
print('MAE:', score)

MAE: 0.10562685093780849
